# Reading in Data

In [25]:
import pandas as pd

In [26]:
# Reading our local files

subscriptions_df = pd.read_csv('../data/input/subscriptions.csv')
techsupportevals_df = pd.read_sas('../data/input/techsupportevals.sas7bdat')
raw_json = pd.read_json('../data/input/reviews.json')
reviews_df = pd.DataFrame.from_records(raw_json['reviews'])


In [27]:
# Connecting to GCS & Snowflake => see the other files

# Getting the data
cust_churn_df = pd.read_parquet("../data/input/gcs/customer_churn_data.parquet")
customers_df = pd.read_sas('../data/input/snowflake/customers.sas7bdat')

## Joining the Data

In [28]:
df = pd.merge(cust_churn_df,customers_df, on='custId',how='inner').drop(columns='custId')

In [29]:
df.columns

Index(['ID', 'LostCustomer', 'regionPctCustomers', 'numOfTotalReturns',
       'wksSinceLastPurch', 'basktPurchCount12Month', 'LastPurchaseAmount',
       'AvgPurchaseAmount12', 'AvgPurchaseAmountTotal', 'intAdExposureCount12',
       'intAdExposureCount36', 'intAdExposureCountAll', 'socialMediaAdCount12',
       'socialMediaAdCount36', 'socialMediaAdCountAll',
       'totalNumProdPurchased', 'custInitiatedContacts', 'avgDiscountValue12',
       'customersales', 'wksSinceFirstPurch', 'reviewId', 'DemHomeOwnerCode',
       'customerGender', 'EstimatedIncome', 'regionMedHomeVal',
       'customerSubscrCode', 'birthDate'],
      dtype='object')

In [30]:
df = df.merge(subscriptions_df, on='customerSubscrCode',how='inner').drop(columns='customerSubscrCode')

In [31]:
df = df.merge(techsupportevals_df, on='ID',how='inner')

In [32]:
len(reviews_df)

782

In [33]:
df['reviewId'].isna().sum()

4218

In [34]:
df = df.merge(reviews_df, on='reviewId',how='left').drop(columns='reviewId')

In [35]:
df.head()

,ID,LostCustomer,regionPctCustomers,numOfTotalReturns,wksSinceLastPurch,basktPurchCount12Month,LastPurchaseAmount,AvgPurchaseAmount12,AvgPurchaseAmountTotal,intAdExposureCount12,...,wksSinceFirstPurch,DemHomeOwnerCode,customerGender,EstimatedIncome,regionMedHomeVal,birthDate,customerSubscrStat,techSupportEval,Review_Text,Title
0,9155.0,0.0,43.0,0.0,14.0,10.0,50.0,0.0,55.65,27.0,...,89.0,b'U',b'F',93000.0,59280.0,2004-11-13,Platinum,3.0,NaN,NaN
1,9160.0,0.0,19.0,1.0,19.0,3.0,50.0,62.5,61.90,13.0,...,89.0,b'H',b'M',84000.0,170820.0,1974-03-14,Platinum,3.0,NaN,NaN
2,9163.0,0.0,19.0,0.0,7.0,10.0,50.0,0.0,35.20,32.0,...,148.0,b'H',b'F',140000.0,92430.0,2006-02-22,Platinum,2.0,NaN,NaN
3,9170.0,0.0,33.0,5.0,7.0,2.0,50.0,40.0,34.75,31.0,...,93.0,b'U',b'F',142000.0,53430.0,2006-04-19,Platinum,2.0,NaN,NaN
4,9175.0,0.0,22.0,0.0,6.0,10.0,50.0,0.0,62.95,40.0,...,137.0,b'H',b'F',83000.0,443690.0,2002-02-23,Platinum,2.0,NaN,NaN


In [36]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 29 columns):
 #   Column                  Non-Null Count  Dtype        
---  ------                  --------------  -----        
 0   ID                      5000 non-null   float64      
 1   LostCustomer            5000 non-null   float64      
 2   regionPctCustomers      5000 non-null   float64      
 3   numOfTotalReturns       5000 non-null   float64      
 4   wksSinceLastPurch       5000 non-null   float64      
 5   basktPurchCount12Month  5000 non-null   float64      
 6   LastPurchaseAmount      5000 non-null   float64      
 7   AvgPurchaseAmount12     5000 non-null   float64      
 8   AvgPurchaseAmountTotal  5000 non-null   float64      
 9   intAdExposureCount12    5000 non-null   float64      
 10  intAdExposureCount36    5000 non-null   float64      
 11  intAdExposureCountAll   5000 non-null   float64      
 12  socialMediaAdCount12    5000 non-null   float64      
 13  soc

## Feature Engeneering

In [37]:
df['DemHomeOwnerCode'].value_counts()

DemHomeOwnerCode
b'H'    2756
b'U'    2244
Name: count, dtype: int64

In [38]:
df['demHomeowner'] = df["DemHomeOwnerCode"].map({'U':'Unknown','H':'Homeowner'})
df.drop(columns='DemHomeOwnerCode', inplace=True)

In [39]:
df['demHomeowner'].value_counts()

Series([], Name: count, dtype: int64)

In [40]:
df['birthDate'].head()

0   2004-11-13
1   1974-03-14
2   2006-02-22
3   2006-04-19
4   2002-02-23
Name: birthDate, dtype: datetime64[s]

In [41]:
import numpy as np

df['customerAge'] = ((pd.Timestamp.now()-pd.to_datetime(df['birthDate'])).dt.days / 365.25)

In [42]:
df['customerAge'].head()

0    20.265572
1    50.934976
2    18.989733
3    18.836413
4    22.986995
Name: customerAge, dtype: float64

In [43]:
df['customerAge'] = df['customerAge'].apply(lambda x: int(x) if pd.notnull(x) else np.nan)
df.drop(columns='birthDate', inplace=True)

In [44]:
df['customerAge'].head()

0    20.0
1    50.0
2    18.0
3    18.0
4    22.0
Name: customerAge, dtype: float64

In [45]:
df['AvgPurchasePerAd'] = df['AvgPurchaseAmount12'] / df['intAdExposureCount12']

In [46]:
df.head()

,ID,LostCustomer,regionPctCustomers,numOfTotalReturns,wksSinceLastPurch,basktPurchCount12Month,LastPurchaseAmount,AvgPurchaseAmount12,AvgPurchaseAmountTotal,intAdExposureCount12,...,customerGender,EstimatedIncome,regionMedHomeVal,customerSubscrStat,techSupportEval,Review_Text,Title,demHomeowner,customerAge,AvgPurchasePerAd
0,9155.0,0.0,43.0,0.0,14.0,10.0,50.0,0.0,55.65,27.0,...,b'F',93000.0,59280.0,Platinum,3.0,NaN,NaN,NaN,20.0,0.000000
1,9160.0,0.0,19.0,1.0,19.0,3.0,50.0,62.5,61.90,13.0,...,b'M',84000.0,170820.0,Platinum,3.0,NaN,NaN,NaN,50.0,4.807692
2,9163.0,0.0,19.0,0.0,7.0,10.0,50.0,0.0,35.20,32.0,...,b'F',140000.0,92430.0,Platinum,2.0,NaN,NaN,NaN,18.0,0.000000
3,9170.0,0.0,33.0,5.0,7.0,2.0,50.0,40.0,34.75,31.0,...,b'F',142000.0,53430.0,Platinum,2.0,NaN,NaN,NaN,18.0,1.290323
4,9175.0,0.0,22.0,0.0,6.0,10.0,50.0,0.0,62.95,40.0,...,b'F',83000.0,443690.0,Platinum,2.0,NaN,NaN,NaN,22.0,0.000000


## Saving Our Work

In [47]:
df.to_csv('../data/output/customer_churn_abt_python.csv')

In [48]:
gcs_key = ('../key/gel-sas-reader.json')

bucket_name = 'sas1-learn'

output_filename = 'customer_churn_abt.parquet'
output_path = f'gcs://{bucket_name}/data/{output_filename}'

df.to_parquet(
    output_path,
    engine='pyarrow',
    storage_options={'token':gcs_key}
)

print(f"DataFrame written to {output_path}")

_request non-retriable exception: Invalid Credentials, 401
Traceback (most recent call last):
  File "/workspaces/myfolder/.user-python-packages/lib/python3.11/site-packages/gcsfs/retry.py", line 132, in retry_request
    return await func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/workspaces/myfolder/.user-python-packages/lib/python3.11/site-packages/gcsfs/core.py", line 470, in _request
    validate_response(status, contents, path, args)
  File "/workspaces/myfolder/.user-python-packages/lib/python3.11/site-packages/gcsfs/retry.py", line 119, in validate_response
    raise HttpError(error)
gcsfs.retry.HttpError: Invalid Credentials, 401


HttpError: Invalid Credentials, 401

In [ ]:
from snowflake.connector.pandas_tools import write_pandas

conn = sf.connector.connect(**sf_credential_dict)

success,nchuncks,nrows,output = write_pandas(
    conn=conn,
    df=df,
    table_name='CUSTOMER_CHURN_AB1'
)

conn.close()